# 🔧 02b - Preprocesamiento de Datos

En este notebook se realizarán las transformaciones necesarias para preparar el dataset `train.csv` antes de entrenar modelos de clasificación.

### Objetivos de esta etapa:

- Imputar valores faltantes (`Age`, `Embarked`)
- Codificar variables categóricas (`Sex`, `Embarked`, `Pclass`)
- Transformar distribuciones sesgadas (`Fare`)
- Generar nuevas variables (`FamilySize`, `IsAlone`, `Title`)
- Guardar el dataset procesado para uso posterior

---


In [1]:
# 📥 Cargar los datos originales para preprocesamiento

import pandas as pd

import warnings
warnings.filterwarnings('ignore')
 
# Ruta relativa al archivo original
ruta = "../data/train.csv"
df = pd.read_csv(ruta)

# Mostrar las primeras filas
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 🧼 Imputación de valores faltantes

En esta celda se imputan los valores nulos de las siguientes columnas:

- `Age`: se imputará usando la mediana agrupada por `Pclass` y `Sex`, ya que hay diferencias significativas entre grupos.
- `Embarked`: solo tiene 2 valores faltantes, se imputará con la moda.
- `Cabin`: tiene más del 77% de valores nulos. En lugar de eliminarla completamente, se creará una variable binaria `HasCabin`.

---


In [2]:
# 🧼 Imputación de valores nulos en 'Embarked', 'Age' y creación de 'HasCabin'

# Imputar 'Embarked' con la moda (valor más frecuente)
embarked_moda = df['Embarked'].mode()[0]
df['Embarked'].fillna(embarked_moda, inplace=True)

# Crear nueva columna 'HasCabin': 1 si tiene cabina, 0 si no
df['HasCabin'] = df['Cabin'].notnull().astype(int)

# Imputar 'Age' con la mediana por grupo de 'Pclass' y 'Sex'
df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

# Verificar que no queden nulos
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
HasCabin         0
dtype: int64

---

✅ **Imputación completada correctamente**

- La columna `Embarked` fue imputada con la moda: `S`.
- La columna `Age` fue imputada con la mediana agrupada por `Pclass` y `Sex`.
- Se creó la nueva variable binaria `HasCabin` a partir de la columna `Cabin`.
- No quedan valores nulos en el dataset. ✔️

---


## 🧠 Extracción del título desde la columna `Name`

La columna `Name` contiene información útil que puede ser aprovechada si se extrae el **título honorífico** (como `Mr.`, `Miss.`, `Dr.`, etc.). Estos títulos están correlacionados con edad, sexo, estatus social y pueden mejorar la capacidad predictiva del modelo.

Ejemplo de valor original:


In [3]:
# 🧠 Extraer título del nombre
df['Title'] = df['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)

# Reemplazar variaciones y títulos poco comunes
titulo_mapa = {
    'Mlle': 'Miss',
    'Ms': 'Miss',
    'Mme': 'Mrs',
    'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare',
    'Col': 'Rare', 'Don': 'Rare', 'Dr': 'Rare', 'Major': 'Rare',
    'Rev': 'Rare', 'Sir': 'Rare', 'Jonkheer': 'Rare', 'Dona': 'Rare'
}
df['Title'] = df['Title'].replace(titulo_mapa)

# Ver los títulos resultantes y su frecuencia
df['Title'].value_counts()


Title
Mr              517
Miss            185
Mrs             126
Master           40
Rare             22
the Countess      1
Name: count, dtype: int64

---

## 🔁 Ajuste final de títulos raros

Se detectó que el título `'the Countess'` quedó fuera de la agrupación `Rare`.  
Vamos a actualizar el diccionario y reagrupar todos los títulos poco frecuentes bajo `'Rare'` para mantener coherencia y evitar variables con muy baja frecuencia que podrían afectar el modelo.

---


In [4]:
# 🔁 Reasignar títulos poco frecuentes, incluyendo 'the Countess'

# Actualizar el mapeo
titulo_mapa_actualizado = {
    'Mlle': 'Miss',
    'Ms': 'Miss',
    'Mme': 'Mrs',
    'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare',
    'Col': 'Rare', 'Don': 'Rare', 'Dr': 'Rare', 'Major': 'Rare',
    'Rev': 'Rare', 'Sir': 'Rare', 'Jonkheer': 'Rare',
    'Dona': 'Rare', 'the Countess': 'Rare'
}

# Aplicar el reemplazo nuevamente
df['Title'] = df['Title'].replace(titulo_mapa_actualizado)

# Verificación final
df['Title'].value_counts()


Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64

---

## 🔠 Codificación de variables categóricas

Vamos a convertir variables categóricas en representaciones numéricas, necesarias para alimentar algoritmos de machine learning. En este paso codificaremos:

- `Sex`: binaria (`0=male`, `1=female`)
- `Embarked`: con One-Hot Encoding (`S`, `C`, `Q`)
- `Title`: con One-Hot Encoding (`Mr`, `Miss`, `Mrs`, `Master`, `Rare`)
- `Pclass`: se tratará como ordinal (1, 2, 3)

---


In [5]:
# 🔠 Codificación de variables categóricas

# Codificar 'Sex' como variable binaria: 0 = male, 1 = female
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# One-hot encoding para 'Embarked' y 'Title'
df = pd.get_dummies(df, columns=['Embarked', 'Title'], drop_first=True)

# Verificación del resultado
df.head()

# Listar las columnas
print(df.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'HasCabin', 'Embarked_Q',
       'Embarked_S', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare'],
      dtype='object')


---
## La salida confirma que la codificación fue exitosa:

- `Sex` → binaria (0/1)

- `Embarked_Q` y `Embarked_S` → dummies (implícitamente Embarked_C como base)

- `Title_*`→ dummies para todos los títulos relevantes (excepto Master, que es la base)

---

---

## 🧪 Transformación de variables y generación de nuevas características

En esta etapa realizamos las siguientes transformaciones:

- `Fare_log`: transformación logarítmica para reducir el sesgo positivo de `Fare`.
- `FamilySize`: suma de `SibSp` (hermanos/cónyuge) y `Parch` (padres/hijos), más 1 (el propio pasajero).
- `IsAlone`: variable binaria que indica si el pasajero viaja solo (`1`) o con familia (`0`).

Estas nuevas variables pueden mejorar el poder predictivo del modelo al capturar relaciones familiares y escalas más normalizadas.

---


In [6]:
import numpy as np

# 🧪 Aplicar log(1 + Fare) para reducir sesgo
df['Fare_log'] = np.log1p(df['Fare'])

# 🧬 Crear variable FamilySize
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# 👤 Crear variable IsAlone
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Verificación
df[['Fare', 'Fare_log', 'FamilySize', 'IsAlone']].head()


,Fare,Fare_log,FamilySize,IsAlone
0,7.2500,2.110213,2,0
1,71.2833,4.280593,2,0
2,7.9250,2.188856,1,1
3,53.1000,3.990834,2,0
4,8.0500,2.202765,1,1


---

## ✅ Transformación y generación de nuevas variables

Las siguientes columnas fueron creadas con éxito:

| Variable     | Descripción                                                                 |
|--------------|------------------------------------------------------------------------------|
| `Fare_log`   | Transformación logarítmica de `Fare` usando `log1p`, reduce el sesgo a la derecha |
| `FamilySize` | Suma de `SibSp` + `Parch` + 1 (el pasajero), refleja el tamaño del grupo familiar |
| `IsAlone`    | Variable binaria: `1` si el pasajero viaja solo, `0` si viaja con familia    |

### 🧾 Ejemplo de resultados:

| Fare   | Fare_log | FamilySize | IsAlone |
|--------|----------|------------|---------|
| 7.25   | 2.11     | 2          | 0       |
| 71.28  | 4.28     | 2          | 0       |
| 7.93   | 2.19     | 1          | 1       |

Estas nuevas variables serán utilizadas como parte del conjunto de entrada para los modelos predictivos en el siguiente paso del pipeline.

---


---

## 🧹 Eliminación de columnas innecesarias y guardado del dataset

En este paso se eliminan las columnas que no aportan valor al modelo o que ya fueron transformadas:

- `Name`, `Ticket`, `Cabin`: contienen información redundante o poco estructurada.
- `Fare`, `SibSp`, `Parch`: fueron reemplazadas por `Fare_log`, `FamilySize` e `IsAlone`.

📌 La columna `PassengerId` **se conserva**, ya que será necesaria para construir el archivo `submission.csv`.

El dataset limpio será guardado como `train_clean.csv` en la carpeta `data/`.

---


In [7]:
# 🧹 Eliminar columnas innecesarias (se conserva PassengerId para el submission)
columnas_a_eliminar = ['Name', 'Ticket', 'Cabin', 'Fare', 'SibSp', 'Parch']
df.drop(columns=columnas_a_eliminar, inplace=True)

# Guardar el dataset limpio
ruta_salida = "../data/train_clean.csv"
df.to_csv(ruta_salida, index=False)

print(f"[INFO] Dataset preprocesado guardado en: {ruta_salida}")
print(f"[INFO] Dimensiones finales: {df.shape}")


df.columns.tolist()  # Listar las columnas finales del DataFrame
# Mostrar las primeras filas del DataFrame final    

[INFO] Dataset preprocesado guardado en: ../data/train_clean.csv
[INFO] Dimensiones finales: (891, 15)


['PassengerId',
 'Survived',
 'Pclass',
 'Sex',
 'Age',
 'HasCabin',
 'Embarked_Q',
 'Embarked_S',
 'Title_Miss',
 'Title_Mr',
 'Title_Mrs',
 'Title_Rare',
 'Fare_log',
 'FamilySize',
 'IsAlone']

---

## 🧾 Resultados del preprocesamiento

El dataset limpio ha sido generado correctamente con las siguientes características:

- ✅ Archivo guardado en: `../data/train_clean.csv`
- ✅ Dimensiones finales: **891 registros**, **15 columnas**
- ✅ Valores nulos: **0** en todo el dataset

### 🔤 Variables disponibles para modelado:

| Tipo             | Columnas                                                                                |
|------------------|------------------------------------------------------------------------------------------|
| Identificador    | `PassengerId` (usado para submission)                                                   |
| Objetivo         | `Survived`                                                                              |
| Numéricas        | `Age`, `Fare_log`, `FamilySize`                                                         |
| Binarias         | `Sex`, `HasCabin`, `IsAlone`                                                            |
| Categóricas dummificadas | `Embarked_Q`, `Embarked_S`, `Title_Miss`, `Title_Mr`, `Title_Mrs`, `Title_Rare` |
| Ordinal          | `Pclass`                                                                                |

📌 Este dataset está listo para ser utilizado en el siguiente paso: entrenamiento de modelos.

---

➡️ A continuación: `03_modelo_baseline.ipynb` – donde se entrenará un primer modelo de clasificación y se generará el archivo `submission.csv`.

---


In [9]:
df

,PassengerId,Survived,Pclass,Sex,Age,HasCabin,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Fare_log,FamilySize,IsAlone
0,1,0,3,0,22.0,0,False,True,False,True,False,False,2.110213,2,0
1,2,1,1,1,38.0,1,False,False,False,False,True,False,4.280593,2,0
2,3,1,3,1,26.0,0,False,True,True,False,False,False,2.188856,1,1
3,4,1,1,1,35.0,1,False,True,False,False,True,False,3.990834,2,0
4,5,0,3,0,35.0,0,False,True,False,True,False,False,2.202765,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.0,0,False,True,False,False,False,True,2.639057,1,1
887,888,1,1,1,19.0,1,False,True,True,False,False,False,3.433987,1,1
888,889,0,3,1,21.5,0,False,True,True,False,False,False,3.196630,4,0
889,890,1,1,0,26.0,1,False,False,False,True,False,False,3.433987,1,1
